In [17]:
# --- Imports and Setup ---
import pandas as pd
import numpy as np
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import optuna
import time
import gc
import warnings
warnings.filterwarnings('ignore')

In [19]:
print("Loading data from your local S: drive...")

try:
    # --- Using the correct folder name 'Kaggle-Fertilizer-Project' ---
    train_path = r'S:\Projects\Kaggle-Fertilizer-Project\data\train.csv'
    test_path = r'S:\Projects\Kaggle-Fertilizer-Project\data\test.csv'
    original_path = r'S:\Projects\Kaggle-Fertilizer-Project\data\Fertilizer Prediction.csv'

    train_df_original = pd.read_csv(train_path)
    test_df_original = pd.read_csv(test_path)
    external_df = pd.read_csv(original_path)

    # Combine all training data
    train_df = pd.concat([train_df_original, external_df], ignore_index=True)
    test_df = test_df_original.copy()
    
    print("All data files loaded successfully!")
    
except FileNotFoundError as e:
    print(f"ERROR: A file was not found. Please double-check your folder and file names.")
    print(f"The path we tried was: {e.filename}")
    raise e


Loading data from your local S: drive...
All data files loaded successfully!


In [20]:
# Cell 3: Generate Base Model Predictions
# This part trains your champion models across 5 folds to get reliable predictions.
# You only need to run this once.

print("\n--- Generating Base Model Predictions (Out-of-Fold) ---")

# --- Champion Model Parameters ---
xgb_params = {
    'device': 'cuda', 'objective': 'multi:softprob', 'eval_metric': 'mlogloss', 'seed': 42,
    'max_depth': 12, 'colsample_bytree': 0.467, 'subsample': 0.86, 'learning_rate': 0.03,
    'gamma': 0.26, 'reg_alpha': 2.7, 'reg_lambda': 1.4,
    'enable_categorical': True, 'num_class': len(y.unique())
}
num_boost_round = 4000

cat_params = {
    'task_type': 'GPU', 'objective': 'MultiClass', 'eval_metric': 'MultiClass', 'verbose': 0, 'random_seed': 42,
    'iterations': 2745, 'learning_rate': 0.0411, 'depth': 5,
    'l2_leaf_reg': 7.96, 'subsample': 0.892, 'bootstrap_type': 'Bernoulli'
}

# --- CV and Prediction Generation ---
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
num_classes = len(y.unique())

oof_preds_xgb = np.zeros((len(X), num_classes))
test_preds_xgb = np.zeros((len(X_test), num_classes))
oof_preds_cat = np.zeros((len(X), num_classes))
test_preds_cat = np.zeros((len(X_test), num_classes))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n===== FOLD {fold + 1} / {N_SPLITS} =====")
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    # --- XGBoost ---
    print("  - Training XGBoost...")
    dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
    dval = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)
    dtest = xgb.DMatrix(X_test, enable_categorical=True)
    xgb_model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=num_boost_round, evals=[(dval, 'eval')], early_stopping_rounds=150, verbose_eval=False)
    oof_preds_xgb[val_idx] = xgb_model.predict(dval, iteration_range=(0, xgb_model.best_iteration))
    test_preds_xgb += xgb_model.predict(dtest, iteration_range=(0, xgb_model.best_iteration)) / N_SPLITS

    # --- CatBoost ---
    print("  - Training CatBoost...")
    cat_model = cb.CatBoostClassifier(**cat_params)
    cat_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
    oof_preds_cat[val_idx] = cat_model.predict_proba(X_val)
    test_preds_cat += cat_model.predict_proba(X_test) / N_SPLITS
    
    gc.collect()

print("\nBase model predictions generated.")


--- Generating Base Model Predictions (Out-of-Fold) ---

===== FOLD 1 / 5 =====
  - Training XGBoost...
  - Training CatBoost...

===== FOLD 2 / 5 =====
  - Training XGBoost...
  - Training CatBoost...

===== FOLD 3 / 5 =====
  - Training XGBoost...
  - Training CatBoost...

===== FOLD 4 / 5 =====
  - Training XGBoost...
  - Training CatBoost...

===== FOLD 5 / 5 =====
  - Training XGBoost...
  - Training CatBoost...

Base model predictions generated.


In [21]:
# Cell 4: Optimize Ensemble Weights with Optuna

print("\n--- Step 4: Optimizing Ensemble Weights ---")

# Define your scoring function
def calculate_map3(y_true, y_pred_probs):
    def ap3(actual, predicted):
        score = 0.0
        for i, p in enumerate(predicted[:3]):
            if p == actual:
                score += 1.0 / (i + 1.0)
                return score
        return score
    return np.mean([ap3(a, p) for a, p in zip(y_true, np.argsort(-y_pred_probs, axis=1))])

def objective_weights(trial):
    # Suggest weights for each model
    w_xgb = trial.suggest_float("w_xgb", 0.0, 1.0)
    w_cat = trial.suggest_float("w_cat", 0.0, 1.0)
    
    # Blend the OOF predictions with the suggested weights
    weighted_oof_preds = (w_xgb * oof_preds_xgb) + (w_cat * oof_preds_cat)
    
    # Return the MAP@3 score
    return calculate_map3(y.values, weighted_oof_preds)

# Run the Optuna study to find the best weights
study_weights = optuna.create_study(direction='maximize', study_name='Weight_Optimization')
study_weights.optimize(objective_weights, n_trials=100, show_progress_bar=True)

best_weights = study_weights.best_params
print("\nBest weights found:", best_weights)

[I 2025-07-15 03:14:58,843] A new study created in memory with name: Weight_Optimization



--- Step 4: Optimizing Ensemble Weights ---


Best trial: 0. Best value: 0.350433:   1%|          | 1/100 [00:01<01:39,  1.01s/it]

[I 2025-07-15 03:14:59,861] Trial 0 finished with value: 0.3504329411764706 and parameters: {'w_xgb': 0.7640242689270729, 'w_cat': 0.6161492511861133}. Best is trial 0 with value: 0.3504329411764706.


Best trial: 0. Best value: 0.350433:   2%|▏         | 2/100 [00:01<01:33,  1.04it/s]

...

[I 2025-07-15 03:16:27,582] Trial 97 finished with value: 0.35146372549019606 and parameters: {'w_xgb': 0.8209230755336969, 'w_cat': 0.29913614110837594}. Best is trial 78 with value: 0.351593137254902.


Best trial: 78. Best value: 0.351593:  99%|█████████▉| 99/100 [01:29<00:00,  1.11it/s]

[I 2025-07-15 03:16:28,490] Trial 98 finished with value: 0.3510780392156863 and parameters: {'w_xgb': 0.41191062511379295, 'w_cat': 0.2340776212978869}. Best is trial 78 with value: 0.351593137254902.


Best trial: 78. Best value: 0.351593: 100%|██████████| 100/100 [01:30<00:00,  1.10it/s]

[I 2025-07-15 03:16:29,412] Trial 99 finished with value: 0.35144509803921575 and parameters: {'w_xgb': 0.9381372581642947, 'w_cat': 0.11929888896597418}. Best is trial 78 with value: 0.351593137254902.

Best weights found: {'w_xgb': 0.8664966498370847, 'w_cat': 0.19963464587757696}


In [22]:
# Cell 5: Create Final Submission with Optimized Weights
# =============================================================================
print("\n--- Step 5: Creating final submission with optimal weights ---")

# Blend the test set predictions using the best weights found by Optuna
w_xgb = best_weights['w_xgb']
w_cat = best_weights['w_cat']
# Normalize the weights so they sum to 1
total_weight = w_xgb + w_cat
final_ensemble_proba = ((w_xgb / total_weight) * test_preds_xgb) + ((w_cat / total_weight) * test_preds_cat)

# --- Create the submission file ---
top_3_indices = np.argsort(final_ensemble_proba, axis=1)[:, -3:][:, ::-1]
predicted_names_list = [' '.join(le.inverse_transform(indices)) for indices in top_3_indices]
submission_df = pd.DataFrame({'id': test_ids, 'Fertilizer Name': predicted_names_list})
submission_df.to_csv('submission_optimized_ensemble.csv', index=False)

print("\nSUCCESS! Final optimized ensemble submission file created.")
print(submission_df.head())


--- Step 5: Creating final submission with optimal weights ---

SUCCESS! Final optimized ensemble submission file created.
       id      Fertilizer Name
0  750000      DAP 28-28 20-20
1  750001  17-17-17 20-20 Urea
2  750002      20-20 28-28 DAP
3  750003    14-35-14 DAP Urea
4  750004  20-20 10-26-26 Urea
